### Import library

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import csv
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

### Define driver

In [2]:
# wd = webdriver.Firefox()
# wd.get('https://www.tokopedia.com/search?st=product&q=sepatu&navsource=home')

In [3]:
chrome_options = Options()
chrome_options.add_argument('user-agent=Chrome/92.0.4515.159')
chrome_options.add_argument('log-level=2')
driver = webdriver.Chrome('./chromedriver.exe',options=chrome_options)
timeout=10

### Connect with tokopedia url

In [6]:
url = 'https://www.tokopedia.com/search?st=product&q=sepatu%20vans&navsource=home'
driver.get(url)
time.sleep(5)

### Get link product

In [7]:
link_product = []
while len(link_product) < 150:
    try:
        driver.execute_script('window.scrollTo(0,5000);')
        time.sleep(3)
        driver.execute_script('window.scrollTo(0,8000);')
        time.sleep(3)
        driver.execute_script('window.scrollTo(0,10000);')
        time.sleep(3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        products = soup.find('div', attrs={'data-testid':'divSRPContentProducts'})
        for product in soup.find_all('a', attrs={'class':'pcv3__info-content'}):
            product_link = product.get('href')
            if len(product_link) <=500:
                link_product.append(product_link)
        time.sleep(3)
        driver.find_element_by_css_selector("button[aria-label='Halaman berikutnya']").click()
        time.sleep(5)
    except TimeoutException:
        print('failed to get links with query ')

In [8]:
seen = set()
uniq_link=[]
for x in link_product:
    if x not in seen:
        uniq_link.append(x)
        seen.add(x)
len(uniq_link)

180

### Get data product

In [14]:
name = []
price = []
discount = []
rate = []
sold_amount = []
noOfRate = []
shop_name =[]
location = []

for link in link_product:
    try:
        url_product = link
        driver.get(url_product)
        time.sleep(6)
        driver.execute_script('window.scrollTo(0,500);')
        time.sleep(4)
        driver.execute_script('window.scrollTo(0,900);')
        time.sleep(4)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # name
        if soup.find('h1' , class_='css-1wtrxts'):
            name0 = soup.find('h1' , class_='css-1wtrxts')
            name.append(name0.text.strip())
        else:
            name.append(None)
        
        # price
        if soup.find('div', class_='price'):
            price0 = soup.find('div', class_='price')
            price.append(price0.text.strip())
        else:
            price.append('0')
        
        # discond
        if soup.find('span', attrs={'data-testid':'lblPDPDetailDiscountPercentage'}):
            discount0 = soup.find('span', attrs={'data-testid':'lblPDPDetailDiscountPercentage'})
            discount.append(discount0.text.strip())
        else:
            discount.append(None)
        
        #rating 
        if soup.find('span', attrs={'data-testid':'lblPDPDetailProductRatingNumber'}):
            rate0 = soup.find('span', attrs={'data-testid':'lblPDPDetailProductRatingNumber'})
            rate.append(rate0.text.strip())
        else:
            rate.append('0')
        
        #noOfRate
        if soup.find('span', attrs={'data-testid':'lblPDPDetailProductRatingCounter'}):
            noOfRate0 = soup.find('span', attrs={'data-testid':'lblPDPDetailProductRatingCounter'})
            noOfRate.append(noOfRate0.text.strip())
        else:
            noOfRate.append('0')
        
        #sold_amount
        if soup.find('div', attrs={'data-testid':'lblPDPDetailProductSoldCounter'}):
            sold_amount0 = soup.find('div', attrs={'data-testid':'lblPDPDetailProductSoldCounter'})
            sold_amount.append(sold_amount0.text.strip())
        else:
            sold_amount.append('0')
        
        #shop_name
        shop_name0 = soup.find('a', attrs={'data-testid':'llbPDPFooterShopName'})
        if shop_name0.find('h2'):
            shop_name1 = shop_name0.find('h2')
            shop_name.append(shop_name1.text.strip())
        else:
            shop_name.append(None)
        
        #location
        shipment = soup.find_all('div', attrs={'class':'css-1le9c0d'})
        if shipment[0].find('b'):
            location0 = shipment[0].find('b')
            location.append(location0.text.strip())
        else:
            location.append(None)
    except TimeoutException:
        print('failed to get desc with query '+line)

In [204]:
len(link_product[3])

113

In [16]:
df = pd.DataFrame(list(zip(name, shop_name, location, price, discount, sold_amount, rate, noOfRate, link_product)),
                 columns=['product_name', 'shop_name', 'location', 'price', 'discount', 'amount_sold', 'rating', 'no_of_rating', 'link_product'])
df.head()

,product_name,shop_name,location,price,discount,amount_sold,rating,no_of_rating,link_product
0,Sepatu VANS OLD SKOOL hitam putih - 40,NAF SHOES,Kab. Tangerang,Rp55.900,None,Terjual 625,4.7,(324 ulasan),https://www.tokopedia.com/nafshoes888/sepatu-v...
1,SEPATU VANS VAULT OLD SKOOL OG BLACK WHITE GRA...,4SDOnlineshop,Jakarta Selatan,Rp114.000,5%,Terjual 253,4.7,(147 ulasan),https://www.tokopedia.com/4sdonlineshop/sepatu...
2,Sepatu Vans Oldskool/Vans cowok/vans cewek/van...,Azkaputra,Jakarta Utara,Rp140.000,None,Terjual 3.080,4.7,(1.359 ulasan),https://www.tokopedia.com/selpiasti/sepatu-van...
3,Sepatu Vans OldSkool Fullwhite/Putih Pria & Wa...,Arty's Shoes,Kab. Tangerang,Rp70.000,None,Terjual 1.298,4.5,(687 ulasan),https://www.tokopedia.com/artyshoes/sepatu-van...
4,SEPATU VANS OLD SKOOL VAULT OG RAINBOW GREEN G...,4SDOnlineshop,Jakarta Selatan,Rp118.750,5%,Terjual 136,4.8,(71 ulasan),https://www.tokopedia.com/4sdonlineshop/sepatu...


### Save file

In [17]:
df.to_json('vans_tokped.json',orient='index')

In [18]:
df.to_csv('vans_tokped.csv',index=False)